LLM本身無法採取行動，它們只能輸出文本。LangChain 的一個重要用例是創建代理。代理是使用 LLM 作為推理引擎來決定要採取哪些行動以及向其傳遞什麼輸入的系統。執行操作後，結果可以反饋回 LLM，以確定是否需要更多操作，或者是否可以結束。

在本教學中，我們將構建一個可以與搜尋引擎互動的代理。您將能夠向此代理提出問題，觀察它調用搜尋工具，並與之進行對話。

端到端代理
下面的程式碼片段代表了一個功能齊全的代理，它使用 LLM 來決定使用哪些工具。它配備了一個通用搜尋工具，並具有對話記憶功能——這意味著它可以用作多輪聊天機器人。

在本指南的其餘部分，我們將詳細介紹各個組件及其作用——但如果您只想獲取一些程式碼並開始使用，請隨意使用此程式碼！

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite langchain-openai

In [2]:
import getpass
import os
from langchain_openai import ChatOpenAI
from anthropic import Anthropic

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# 替換為你的LANGCHAIN_API_KEY
os.environ["LANGCHAIN_API_KEY"] = "替換為你的LANGCHAIN_API_KEY"

os.environ["OPENAI_API_KEY"] = "替換為你的OPENAI_API_KEY"

os.environ["TAVILY_API_KEY"] ="替換為你的TAVILY_API_KEY"

定義工具
首先，我們需要創建我們想要使用的工具。我們的主要工具選擇將是 [Tavily](https://github.com/langchain-ai/langchain/blob/49dea06af15a110518f7a119260f609a4b5f74fe/docs/docs/tutorials/docs/integrations/tools/tavily_search) —— 一個搜尋引擎。LangChain 內建了一個工具，可以輕鬆地將 Tavily 搜尋引擎作為工具使用。

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1724075718, 'localtime': '2024-08-19 06:55'}, 'current': {'last_updated_epoch': 1724075100, 'last_updated': '2024-08-19 06:45', 'temp_c': 15.2, 'temp_f': 59.4, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 75, 'feelslike_c': 15.2, 'feelslike_f': 59.4, 'windchill_c': 11.9, 'windchill_f': 53.5, 'heatindex_c': 12.4, 'heatindex_f': 54.4, 'dewpoint_c': 11.2, 'dewpoint_f': 52.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 7.6, 'gust_kph': 12.2}}"}, {'url': 'https://www.weathertab.com/en/c/e/08/unit

API Reference:[TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)

您可以通過傳入一個消息列表來調用語言模型。默認情況下，響應是一個 `content` 字串。


In [4]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

API Reference:[HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)

現在，我們可以看看如何讓這個模型能夠進行工具調用。為了實現這一點，我們使用 `.bind_tools` 方法讓語言模型了解這些工具。


In [5]:
model_with_tools = model.bind_tools(tools)

現在我們可以調用模型了。讓我們首先用一個普通的訊息來調用它，看看它是如何響應的。我們可以查看 `content` 字段以及 `tool_calls` 字段。


In [6]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


現在，讓我們嘗試用一些預計會調用工具的輸入來調用它。


In [7]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Sorry, I'm a text-based assistant and currently don't have access to real-time weather data. You can check the weather in San Francisco by using a weather forecasting service or app like the Weather Channel, BBC Weather, or your phone's built-in weather app.
ToolCalls: []


我們可以看到現在沒有文本內容，但有一個工具調用！它希望我們調用 Tavily Search 工具。

這還沒有真正調用該工具 - 它只是告訴我們這樣做。為了實際調用它，我們需要創建我們的代理。

## 創建代理

現在我們已經定義了工具和 LLM，我們可以創建代理了。我們將使用 [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) 來構建代理。目前，我們正在使用高級接口來構建代理，但 LangGraph 的優點是，這個高級接口背後有一個低級、高度可控的 API，以防您想修改代理邏輯。

現在，我們可以使用 LLM 和工具來初始化代理。

請注意，我們傳入的是 `model`，而不是 `model_with_tools`。這是因為 `create_react_agent` 會在內部為我們調用 `.bind_tools` 方法。


In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## 運行代理(Agent)

現在我們可以在一些查詢上運行代理了！請注意，目前這些都是**無狀態**查詢（它不會記住之前的交互）。請注意，代理將在交互結束時返回**最終**狀態（其中包括任何輸入，我們稍後將看到如何僅獲取輸出）。

首先，讓我們看看當不需要調用工具時它是如何響應的：


In [9]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='1adcdfa0-2300-4a85-bf40-cdb7e9f56f78'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd7cc3fc-fb6e-4099-a714-48c3fb265d91-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

為了確切了解幕後發生了什麼（並確保它沒有調用工具），我們可以查看 [LangSmith 的追蹤記錄](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/)。

現在讓我們嘗試一個它應該會調用工具的例子。


In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='c949efa1-c511-4683-9044-466af1f936e2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g0cGcR3iX0sQ2uKN3agjKjY0', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b15d6b87-079f-4ba6-a691-703e4a339150-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_g0cGcR3iX0sQ2uKN3agjKjY0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California

我們可以查看 [LangSmith 追蹤紀錄](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r)，以確保它有效地調用了搜尋工具。

## 串流訊息

我們已經看到如何使用 `.invoke` 調用代理來獲取最終響應。如果代理正在執行多個步驟，這可能需要一段時間。為了顯示中間進度，我們可以在訊息發生時將它們串流傳輸回來。


In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hjsd39wYncqNeNia91ecuJDZ', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2325a5ad-0f54-426f-b993-471eb0d3be12-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_Hjsd39wYncqNeNia91ecuJDZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of Ameri

## 串流 token

除了串流傳輸回消息外，串流傳輸回 token 也很有用。我們可以使用 `.astream_events` 方法來做到這一點。


In [12]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


I|'m| sorry|,| I| am| not| currently| able| to| provide| real|-time| data| such| as| weather| updates|.| Please| use| a| trusted| weather| reporting| service| for| the| most| accurate| information|.|

雖然這個狀態表示這個tool目前不能用來抓取實時天氣的資料，不過也證明了起碼交互式有建立起來的。

## 添加記憶功能

如前所述，此代理是無狀態的。這意味著它不記得先前的互動。為了賦予它記憶功能，我們需要傳入一個檢查點（checkpointer）。當傳入檢查點時，我們還必須在調用代理時傳入一個 `thread_id`（以便它知道要從哪個線程/對話恢復）。


In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 90, 'total_tokens': 101}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-438e6eac-020b-4d27-a464-c17bb086c350-0', usage_metadata={'input_tokens': 90, 'output_tokens': 11, 'total_tokens': 101})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 113, 'total_tokens': 119}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0fa20512-9b6b-412c-8ed3-29a7a9610181-0', usage_metadata={'input_tokens': 113, 'output_tokens': 6, 'total_tokens': 119})]}}
----


如果我想開始一個新的對話，我只需更改使用的 `thread_id` 即可。


In [15]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 86, 'total_tokens': 125}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-605ec129-3f5c-4c8c-a570-1751d9037acd-0', usage_metadata={'input_tokens': 86, 'output_tokens': 39, 'total_tokens': 125})]}}
----


## 結語

這就是本快速入門的全部內容！在這裡，我們介紹了如何創建一個簡單的代理。然後，我們展示了如何串流傳輸回回應——不僅是中間步驟，還包括 token！我們還添加了記憶功能，讓您可以與它們進行對話。代理是一個複雜的主題，還有很多東西要學！

欲了解更多關於代理的資訊，請查看 [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) 文檔。它有自己的一套概念、教程和操作指南。
